In [1]:
import numpy as np
import numpy.linalg as LA
import meshplot as mp
from IPython.display import JSON as DJSON
from IPython.display import clear_output
from pspart import Part
from pspart import NNHash
import os
import pandas as ps
from mate_proposals import mate_proposals, homogenize_frame
from scipy.spatial.transform import Rotation as R
#import meshplot as mp
import onshape.brepio as brepio
import time
from automate.data.data import UniformMateData
import torch
import random
from automate.lightning_models.simplified import SimplifiedJointModel
from automate.data.transforms import compose

In [5]:
datapath = '/projects/grail/benjones/cadlab'

In [6]:
weights_path='/projects/grail/benjones/cadlab/dalton_lightning_logs/real_all_fn_args_amounts_sum_directedhybridgcn12/version_0/checkpoints/epoch=46-val_auc=0.666113.ckpt'
model = SimplifiedJointModel.load_from_checkpoint(weights_path, map_location=torch.device('cpu'))
transforms = model.transforms
del model

/fast/jamesn8/anaconda3/envs/torch3/lib/python3.9/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Metric` was deprecated since v1.3.0 in favor of `torchmetrics.metric.Metric`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


In [7]:
transform = compose(*(transforms[::-1]))

In [8]:
name = '/fast/jamesn8/assembly_data/assembly_data_with_transforms_all.h5'
assembly_df = ps.read_hdf(name,'assembly')
mate_df = ps.read_hdf(name,'mate')
part_df = ps.read_hdf(name,'part')
mate_df['MateIndex'] = mate_df.index
part_df['PartIndex'] = part_df.index

In [9]:
mate_df.set_index('Assembly', inplace=True)
part_df.set_index('Assembly', inplace=True)    

In [10]:
with open('fully_connected_moving_no_multimates.txt','r') as f:
    set_E_indices = [int(l.rstrip()) for l in f.readlines()]

In [11]:
mate_types = [
            'PIN_SLOT',
            'BALL',
            'PARALLEL',
            'SLIDER',
            'REVOLUTE',
            'CYLINDRICAL',
            'PLANAR',
            'FASTENED'
        ]

In [12]:
def global_bounding_box(parts, transforms=None):
    if transforms is None:
        allpoints = [part.V for part in parts if part.V.shape[0] > 0]
    else:
        allpoints = [(tf[:3,:3] @ part.V.T + tf[:3,3,np.newaxis]).T for tf, part in zip(transforms, parts) if part.V.shape[0] > 0]
    if len(allpoints) == 0:
        return None
    minPt = np.array([points.min(axis=0) for points in allpoints]).min(axis=0)
    maxPt = np.array([points.max(axis=0) for points in allpoints]).max(axis=0)
    return np.vstack([minPt, maxPt])

In [13]:
def apply_transform(tf, v, is_points=True):
    "Apply the 4-matrix `tf` to a vector or column-wise matrix of vectors. If is_points, also add the translation."
    v_trans = tf[:3,:3] @ v
    if is_points:
        if v.ndim==1:
            v_trans += tf[:3,3]
        elif v.ndim==2:
            v_trans += tf[:3,3,np.newaxis]
    return v_trans

In [14]:
def cs_to_origin_frame(cs):
    return cs[:3,3], cs[:3,:3]

In [15]:
def cs_from_origin_frame(origin, frame):
    cs = np.identity(4, np.float64)
    cs[:3,:3] = frame
    cs[:3,3] = origin
    return cs

In [30]:
#start_index = 1243
start_index = 0

outpath = '/fast/jamesn8/assembly_data/mate_torch/data'
statspath = '/fast/jamesn8/assembly_data/mate_torch/stats'
logfile = os.path.join(statspath, 'log.txt')
def LOG(st):
    with open(logfile,'a') as logf:
        logf.write(st + '\n')
epsilon_rel = 0.001
max_groups = 10
max_mcs = 10000
max_mc_pairs = 100000
stride = 1000
debug_run=False
save_stats=True
last_mate_ckpt = 0
last_ckpt = 0

all_stats = []
mate_stats = []
processed_indices = []
mate_indices = []
invalid_part_paths_and_transforms = []

run_start_time = time.time()
for num_processed,ind in enumerate(set_E_indices[start_index:]):
    stats = dict()
    curr_mate_stats = []
    #clear_output(wait=True)
    display(f'num_processed: {num_processed+start_index}/{len(set_E_indices[start_index:])}')

    #1. spatially hash all MCFs (cache hash maps for each part for re-use with individual mates)
    #2. for all mates, ensure that each MCF is represented (keep track of closest/equivalent MCFs, log percentage of assemblies for which this holds)
    #3. get proposals, edit appropriate ones to true based on equivalence class computed per mated pair of parts (taking outer product of equivalent MCs on left and right)

    print(assembly_df.loc[ind,"AssemblyPath"])
    LOG(f'{num_processed+start_index}/{len(set_E_indices[start_index:])}: processing {assembly_df.loc[ind,"AssemblyPath"]} at {time.time()-run_start_time}')
    
    part_subset = part_df.loc[ind]
    mate_subset = mate_df.loc[ind]
    if mate_subset.ndim == 1:
        mate_subset = ps.DataFrame([mate_subset], columns=mate_subset.keys())
    
    parts = []
    part_caches = []
    part_paths = []
    transforms = []
    mcf_hashes = [] #hashes of normalized mcfs ([origin/maxdim, quat])
    mco_hashes = [] #hashes of origins only
    mc_frames_all = [] #[[origin, homogenized rotmatrix]] for each mc for each part
    occ_to_index = dict()
    
    for j in range(part_subset.shape[0]):
        path = os.path.join(datapath, 'data/models', *[part_subset.iloc[j][k] for k in ['did','mv','eid','config']], f'{part_subset.iloc[j]["PartId"]}.xt')
        assert(os.path.isfile(path))
        with open(path) as partf:
            part_str = partf.read()
            part = Part(part_str)
            part_paths.append(path)
            part_caches.append(part_str)
            parts.append(part)
            tf = part_subset.iloc[j]['Transform']
            transforms.append(tf)
            occ_to_index[part_subset.iloc[j]['PartOccurrenceID']] = j
    
    
    bbox = global_bounding_box(parts, transforms)
    if bbox is None:
        LOG('skipping due to no geometry')
        continue
    minPt, maxPt = bbox
    
    median = bbox.mean(axis=0)
    dims = maxPt - minPt
    maxdim = max(dims)
    #maxdim = max([(part.bounding_box()[1]-part.bounding_box()[0]).max() for part in parts])
    threshold = maxdim * epsilon_rel
    
    p_normalized = np.identity(4, dtype=float)
    p_normalized[:3,3] = -median
    p_normalized[3,3] = maxdim #todo: figure out if this is double the factor

    norm_matrices = [p_normalized @ tf for tf in transforms]
    normalized_parts = [Part(cached_part, mat) for cached_part, mat in zip(part_caches, norm_matrices)]
    num_invalid_parts = 0
    for npart, path, mat in zip(normalized_parts, part_paths, norm_matrices):
        if not npart.valid:
            LOG(f'invalid part {path}')
            LOG(f'transform:\n{mat}')
            num_invalid_parts += 1
    stats['num_invalid_transformed_parts'] = num_invalid_parts
    stats['num_normalized_parts_with_different_graph_size'] = 0
    for npart, part in zip(normalized_parts, parts):
        if npart.num_topologies != part.num_topologies:
            stats['num_normalized_parts_with_different_graph_size'] += 1
    
    total_mcs = sum([len(part.all_mate_connectors) for part in parts])
    stats['total_mates'] = mate_subset.shape[0]
    stats['total_parts'] = len(parts)
    stats['maxdim'] = maxdim
    stats['total_mcs'] = total_mcs

    for j in range(len(parts)):
        part = parts[j]
        tf = transforms[j]
        mc_frames = []
        mc_frames_normalized = []
        mc_origins = []
        for mc in part.all_mate_connectors:
            cs = mc.get_coordinate_system()
            origin, frame = cs_to_origin_frame(tf @ cs)
            frame_homogenized = homogenize_frame(frame, z_flip_only=True)
            #all_points.append(origin)
            rot = R.from_matrix(frame_homogenized).as_quat()
            mc_origins.append(origin)
            mc_frames.append((origin, frame_homogenized))
            mc_frames_normalized.append(np.concatenate([origin/maxdim, rot]))
        mc_frames_all.append(mc_frames)
        frame_hash = NNHash(mc_frames_normalized, 7, epsilon_rel)
        origin_hash = NNHash(mc_origins, 3, threshold)
        #frame_hash = NNHash([mc_frame[:3] for mc_frame in mc_frames], 3, threshold)
        mcf_hashes.append(frame_hash)
        mco_hashes.append(origin_hash)

    stats['invalid_frames'] = 0
    stats['invalid_mates'] = 0
    stats['invalid_coincident_origins'] = 0
    stats['invalid_permuted_z'] = 0
    
    mate_matches = [] #list of (left MC IDs, right MC Ids) based on the type of mate
    part_pair_to_mate = dict()

    #all_points = np.array(all_points)
    #p = mp.plot(all_points)
    mate_invalids = []
    for j in range(mate_subset.shape[0]):
        matches = [set(), set()]
        m_stats = dict()
        part_indices = []
        mate_invalid = False
        for i in range(2):
            occId = mate_subset.iloc[j][f'Part{i+1}']
            partIndex = occ_to_index[occId]
            part_indices.append(partIndex)
            assert(part_subset.iloc[partIndex]['PartOccurrenceID'] == occId)
            origin_local = mate_subset.iloc[j][f'Origin{i+1}']
            frame_local = mate_subset.iloc[j][f'Axes{i+1}']
            cs = cs_from_origin_frame(origin_local, frame_local)
            origin, frame = cs_to_origin_frame(transforms[partIndex] @ cs)
            frame_homogenized = homogenize_frame(frame, z_flip_only=True)
            rot = R.from_matrix(frame_homogenized).as_quat()
            mc_frame_normalized = np.concatenate([origin/maxdim, rot])
            neighbors = mcf_hashes[partIndex].get_nearest_points(mc_frame_normalized)

            for n in neighbors:
                matches[i].add(n)
            
            b_invalid = False
            b_num_matches = len(neighbors)
            b_invalid_coincident_origins = False
            b_invalid_permuted_z = False
            
            #compute statistics for the case where there are no matching mcs
            if len(neighbors) == 0:
                b_invalid = True
                stats['invalid_frames'] += 1
                if not mate_invalid:
                    stats['invalid_mates'] += 1
                    mate_invalid = True
                
                origin_neighbors = mco_hashes[partIndex].get_nearest_points(origin)
                if len(origin_neighbors) > 0:
                    b_invalid_coincident_origins = True
                    stats['invalid_coincident_origins'] += 1
                    n = next(iter(origin_neighbors))
                    
                    #detect whether allowing Z permutations would make this frame match
                    c_frame = mc_frames_all[partIndex][n][1]
                    c_frame_homogenized = homogenize_frame(c_frame, z_flip_only=False)
                    mate_frame_homogenized = homogenize_frame(frame, z_flip_only=False)
                    dist = LA.norm(c_frame_homogenized - mate_frame_homogenized)

                    if dist < threshold:
                        b_invalid_permuted_z = True
                        stats['invalid_permuted_z'] += 1
            else:
                mateType = mate_subset.iloc[j]['Type']
                mc_frames = mc_frames_all[partIndex]
                for k,c_origin_frame in enumerate(mc_frames):
                    c_origin, c_frame = c_origin_frame
                    axisdist = LA.norm(c_frame[:,2] - frame_homogenized[:,2])
                    if axisdist < epsilon_rel:
                        projdist = np.inf
                        if mateType == 'CYLINDRICAL' or mateType == 'SLIDER':
                            c_origin_proj = c_origin @ frame_homogenized[:,:2]
                            origin_proj = origin @ frame_homogenized[:,:2]
                            projdist = LA.norm(c_origin_proj - origin_proj)
                        elif mateType == 'PLANAR' or mateType == 'PARALLEL':
                            c_origin_proj = c_origin.dot(frame_homogenized[:,2])
                            origin_proj = origin.dot(frame_homogenized[:,2])
                            projdist = abs(c_origin_proj - origin_proj)
                        if projdist < epsilon_rel:
                            matches[i].add(k)
                    
            
            m_stats[f'invalid_frame_{i}'] = b_invalid
            m_stats[f'invalid_frame_{i}_coincident_origins'] = b_invalid_coincident_origins
            m_stats[f'invalid_frame_{i}_permuted_z'] = b_invalid_permuted_z
            m_stats[f'matches_frame_{i}'] = b_num_matches
            m_stats[f'extra_matches_frame_{i}'] = len(matches[i]) - b_num_matches
        m_stats['type'] = mate_subset.iloc[j]['Type']
        m_stats['truncated_mc_pairs'] = False
        curr_mate_stats.append(m_stats)
        mate_indices.append(mate_subset.iloc[j]['MateIndex'])
        mate_invalids.append(mate_invalid)
        mate_matches.append(matches)
        part_indices = tuple(sorted(part_indices))
        part_pair_to_mate[part_indices] = j#mate_subset.iloc[j]['Type']
    
    if total_mcs <= max_mcs and stats['num_normalized_parts_with_different_graph_size'] == 0:
        stats['false_part_pairs'] = 0
        stats['missed_part_pairs'] = 0
        stats['missed_mc_pairs'] = 0
        #find assembly-level normalization matrix
        
        #DEBUG
#         bbox = global_bounding_box(normalized_parts)
#         print(f'maxdim: {maxdim}')
#         print(f'bbox median: {bbox.mean(axis=0)}, dims: {bbox[1,:]-bbox[0,:]}')
#         p = mp.plot(normalized_parts[0].V, normalized_parts[0].F)
#         for part in normalized_parts[1:]:
#             p.add_mesh(part.V, part.F)
        
        
        #find match proposals
        start = time.time()
        proposals = mate_proposals(list(zip(transforms, parts)), epsilon_rel=epsilon_rel, max_groups=max_groups)
        end = time.time()
        stats['num_proposals'] = len(proposals)
        stats['proposal_time'] = end-start
        
        #initialize pairs based on proposals
        part_proposals = dict()
        for proposal in proposals:
            part_pair = proposal[:2]
            if part_pair not in part_proposals:
                mc_pair_dict = dict()
                part_proposals[part_pair] = mc_pair_dict
            else:
                mc_pair_dict = part_proposals[part_pair]
            mc_pair_dict[proposal[2:]] = -1 #mate type

        #populate pairs with labels
        #print('populating pairs with labels')
        part_pair_found=False
        mc_pair_found=False
        for j in range(mate_subset.shape[0]):
            if not mate_invalids[j]:
                mate_type = mate_subset.iloc[j]['Type']
                partIds = [occ_to_index[mate_subset.iloc[j][f'Part{i+1}']] for i in range(2)]
                matches = mate_matches[j]

                if partIds[0] > partIds[1]:
                    partIds.reverse()
                    matches = matches.copy()
                    matches.reverse()
                partIds = tuple(partIds)

                if partIds in part_proposals:
                    part_pair_found=True
                    mc_pair_dict = part_proposals[partIds]
                    for index1 in matches[0]:
                        for index2 in matches[1]:
                            mc_pair = index1, index2
                            if mc_pair in mc_pair_dict:
                                mc_pair_found=True
                                mc_pair_dict[mc_pair] = mate_types.index(mate_type)
            if not part_pair_found:
                stats['missed_part_pairs'] += 1
            if not mc_pair_found:
                stats['missed_mc_pairs'] += 1
            curr_mate_stats[j]['part_pair_found'] =  part_pair_found           
            curr_mate_stats[j]['mc_pair_found'] =  mc_pair_found     
        
        #create data object for each part pair
        #print('creating data object')
        for part_pair in part_proposals:
            if normalized_parts[part_pair[0]].valid and normalized_parts[part_pair[1]].valid:
                mateIndex = -1
                if part_pair in part_pair_to_mate:
                    mateIndex = part_pair_to_mate[part_pair]
                    mateType = mate_subset.iloc[mateIndex]['Type']
                else:
                    mateType='FASTENED'
                    stats['false_part_pairs'] += 1
                mc_pairs = part_proposals[part_pair]

                if len(mc_pairs) > max_mc_pairs:
                    curr_mate_stats[mateIndex]['truncated_mc_pairs'] = True
                    mc_pairs_final=[]
                    mc_pairs_false=[]
                    for pair in mc_pairs:
                        if mc_pairs[pair] >= 0:
                            mc_pairs_final.append(pair)
                        else:
                            mc_pairs_false.append(pair)
                    N_true = len(mc_pairs_final)
                    N_remainder = max_mc_pairs - N_true
                    random.shuffle(mc_pairs_false)
                    for pair in mc_pairs_false[:N_remainder]:
                        mc_pairs_final.append(pair)
                else:
                    mc_pairs_final = mc_pairs

                part1 = parts[part_pair[0]]
                part2 = parts[part_pair[1]]
                or1, loc1, inf1 = part1.get_onshape_def_from_mc(part1.all_mate_connectors[0])
                or2, loc2, inf2 = part2.get_onshape_def_from_mc(part2.all_mate_connectors[0])

                data = UniformMateData(
                    normalized_parts[part_pair[0]],
                    or1,
                    loc1,
                    inf1,
                    norm_matrices[part_pair[0]],
                    normalized_parts[part_pair[1]],
                    or2,
                    loc2,
                    inf2,
                    norm_matrices[part_pair[1]],
                    mateType
                )
                data = transform(data)
                data.mc_pairs = torch.empty((6, len(mc_pairs_final)), dtype=torch.int64)
                data.mc_pair_labels = torch.zeros(len(mc_pairs_final), dtype=torch.float32)
                all_mcs = [parts[partIndex].all_mate_connectors for partIndex in part_pair]
                for k,pair in enumerate(mc_pairs_final):
                    type_index = mc_pairs[pair]
                    if type_index >= 0:
                        data.mc_pair_labels[k] = 1
                    mcs = [mc_list[mcIndex] for mc_list, mcIndex in zip(all_mcs, pair)]
                    col = torch.tensor([mcs[0].orientation_inference.topology_ref, mcs[0].location_inference.topology_ref, mcs[0].location_inference.inference_type.value,
                          mcs[1].orientation_inference.topology_ref, mcs[1].location_inference.topology_ref, mcs[1].location_inference.inference_type.value], dtype=torch.int)
                    data.mc_pairs[:,k] = col
                #dataname = f'{assembly_df.loc[ind,"AssemblyPath"]}-{part_subset.iloc[part_pair[0]]["PartOccurrenceID"].replace("/","_")}-{part_subset.iloc[part_pair[1]]["PartOccurrenceID"].replace("/","_")}.dat'
                assert(data.node_types_g1.shape[0] == part1.num_topologies)
                assert(data.node_types_g2.shape[0] == part2.num_topologies)
                assert(data.mc_pairs[0].max().item() < part1.num_topologies)
                assert(data.mc_pairs[1].max().item() < part1.num_topologies)
                assert(data.mc_pairs[3].max().item() < part2.num_topologies)
                assert(data.mc_pairs[4].max().item() < part2.num_topologies)
                if not debug_run:
                    dataname = f'{ind}-{part_subset.iloc[part_pair[0]]["PartIndex"]}-{part_subset.iloc[part_pair[1]]["PartIndex"]}.dat'
                    torch_datapath = os.path.join(outpath, dataname)
                    torch.save(data, torch_datapath)
                    del data
            
    for stat in curr_mate_stats:
        mate_stats.append(stat)
    all_stats.append(stats)
    processed_indices.append(ind)
    
    if save_stats and (num_processed+1) % stride == 0:
        
        stat_df_mini = ps.DataFrame(all_stats[last_ckpt:], index=processed_indices[last_ckpt:])
        mate_stat_df_mini = ps.DataFrame(mate_stats[last_mate_ckpt:], index=mate_indices[last_mate_ckpt:])
        stat_df_mini.to_parquet(os.path.join(statspath, f'stats_{num_processed + start_index}.parquet'))
        mate_stat_df_mini.to_parquet(os.path.join(statspath, f'mate_stats_{ind}.parquet'))
        last_mate_ckpt = len(mate_indices)
        last_ckpt = len(processed_indices)

'num_processed: 0/20845'

0e2de88aa724d660e4093564_52ea6bfdd317472a99efcffa_316901caae65ff373c71ef88


'num_processed: 1/20845'

20acdbfb860686db5801f7db_5676ec505c49f9284cf6f810_69f3bde151e038c3d9a8d55c


'num_processed: 2/20845'

...

'num_processed: 2673/20845'

5f8a7f90b76905a81e8ba96a_e218293b8f9892cbf775c99f_4f42df63777c625839304997


'num_processed: 2674/20845'

8bd56e632044ba3842b52cb7_e1e38cb4700f971d1beaed83_dff6d4adba6943cfb7e0c66a


KeyboardInterrupt: 

In [ ]:
num_processed

In [24]:
data

UniformMateData(amounts_data_g1=[167, 1], amounts_data_g2=[252, 1], brep_directed_edge_index_g1=[2, 284], brep_directed_edge_index_g2=[2, 450], brep_edge_types_g1=[284], brep_edge_types_g2=[450], brep_edges_g1=[2, 284], brep_edges_g2=[2, 450], face_edge_edge_index_g1=[2, 128], face_edge_edge_index_g2=[2, 200], face_edge_edge_map_g1=[64, 1], face_edge_edge_map_g2=[100, 1], face_edge_edge_types_g1=[64], face_edge_edge_types_g2=[100], face_edge_edges_g1=[2, 64], face_edge_edges_g2=[2, 100], face_edge_topology_map_g1=[2, 284], face_edge_topology_map_g2=[2, 450], left_mc_individual_labels=[18], left_mc_individual_refs=[4, 18], mate_type=[1], mc_pair_labels=[8179], mc_pair_type=[162], mc_pairs=[6, 8179], mc_pairs_left_cs=[162, 16], mc_pairs_right_cs=[162, 16], node_types_g1=[167], node_types_g2=[252], num_G_edges_g1=64, num_G_edges_g2=100, num_G_faces_g1=31, num_G_faces_g2=46, num_G_loops_g1=36, num_G_loops_g2=50, num_G_verts_g1=36, num_G_verts_g2=56, p_normalized_g1=[4, 4], p_normalized_g2=

In [26]:
data.node_types_g2.shape

torch.Size([252])

In [27]:
part2

<Part with 280 nodes and 502 links>

In [28]:
normalized_parts[part_pair[1]]

<Part with 252 nodes and 450 links>

# Tmp space

In [31]:
from automate.data.saved_dataset import SavedDataset
from torch_geometric.data import Batch
from automate.data.transforms import select_full_relations
import torch
from torch_geometric.data.dataloader import DataLoader

In [32]:
indexfile = '/fast/jamesn8/assembly_data/mate_torch_norm_match/index_partial2_filtered_train.txt'
dataset = SavedDataset(indexfile, '/fast/jamesn8/assembly_data/mate_torch_norm_match/data')

In [35]:
follow_batch=[
            'node_types_g1', 'node_types_g2',
            'mc_index_g1', 'mc_index_g2',
            'mc_pair_labels',
            'left_mc_individual_labels', 'right_mc_individual_labels'
        ]
dataloader = DataLoader(dataset, batch_size=4, follow_batch=follow_batch)

In [40]:
pos_counts=0
neg_counts=0
for i,data in enumerate(dataloader):
    pos_counts += data.mc_pair_labels.nonzero().size(0)
    neg_counts += (data.mc_pair_labels == 0).nonzero().size(0)
    if i > 10000:
        break

In [41]:
pos_counts

2784086

In [42]:
neg_counts

69725667

In [43]:
neg_counts/pos_counts

25.044365368023833